<!-- ---
title: How to do time profiling
downloads: true
sidebar: true
tags:
  - time-profiling
  - BasicTimeProfiler
  - Timer
  - HandlersTimeProfiler
--- -->
# How to do time profiling

This example demonstrates how you can get the time breakdown for:
- All epochs during training
- Individual `Events`
- All `Handlers` correspoding to an `Event`
- Individual `Handlers`
- Data loading and Data processing.

In this example, we will be using a ResNet18 model on the MNIST dataset. The base code is the same as used in the Getting Started Guide.

## Basic Setup

In [22]:
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.models import resnet18
from torchvision.transforms import Compose, Normalize, ToTensor

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import Timer
from ignite.contrib.handlers import BasicTimeProfiler, HandlersTimeProfiler

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.model = resnet18(num_classes=10)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        return self.model(x)


model = Net().to(device)

data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

train_loader = DataLoader(
    MNIST(download=True, root=".", transform=data_transform, train=True),
    batch_size=128,
    shuffle=True,
)

val_loader = DataLoader(
    MNIST(download=True, root=".", transform=data_transform, train=False),
    batch_size=256,
    shuffle=False,
)

optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

We attach two handlers to the `trainer` to print out the metrics ([`Accuracy`](https://pytorch.org/ignite/generated/ignite.metrics.Accuracy.html#accuracy) and [`Loss`](https://pytorch.org/ignite/generated/ignite.metrics.Loss.html#loss)) of the train and validation dataset at the end of every epoch.

In [49]:
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(
    model, metrics={"accuracy": Accuracy(), "loss": Loss(criterion)}, device=device
)


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    print(
        f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}"
    )


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print(
        f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}"
    )

## Custom profiling using Timer

At the lowest level of abstraction, we provide [`Timer()`](https://pytorch.org/ignite/generated/ignite.handlers.timing.Timer.html#timer) to calculate the time between any set of events. Below, we will calculate the total training time and average time taken by a single epoch.

In [50]:
timer = Timer(average=True)
timer.attach(trainer, step=Events.EPOCH_COMPLETED)

We set `average=True` to automatically divide the total time the `timer` was running by an internal counter whoose value is decided by the [`step`](https://pytorch.org/ignite/generated/ignite.handlers.timing.Timer.html#ignite.handlers.timing.Timer.step) method. Here we have passed the event `EPOCH_COMPLETED` to `step` so that the internal counter will be incremented every time an epoch is completed. This way `timer.value()` will return `timer.total` divided by the number of epochs which is equal to the average time taken by a single epoch. Finally we attach `timer` to `trainer`.  

In [51]:
trainer.run(train_loader, max_epochs=2)

print(
    f"Total time: {timer.total:.2f}, Average time taken by a single epoch: {timer.value():.2f}"
)

Training Results - Epoch[1] Avg accuracy: 0.97 Avg loss: 0.11
Validation Results - Epoch[1] Avg accuracy: 0.97 Avg loss: 0.10
Training Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.05
Validation Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.04
Total time: 93.11, Average time taken by a single epoch: 46.55


## Using `State` of Events

If we just want to print the time taken after every epoch and the total time for training we can simply use the `trainer`'s [`State`](https://pytorch.org/ignite/generated/ignite.engine.events.State.html#ignite.engine.events.State). We attach two separate handlers fired when an epoch is completed and when the training is completed to log the time returned by `trainer.state.times`.

In [52]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_epoch_time():
    print(
        f"Epoch {trainer.state.epoch}, Time Taken : {trainer.state.times['EPOCH_COMPLETED']}"
    )


@trainer.on(Events.COMPLETED)
def log_total_time():
    print(f"Total Time: {trainer.state.times['COMPLETED']}")

In [53]:
trainer.run(train_loader, max_epochs=2)

Training Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.03
Validation Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.03
Epoch 1, Time Taken : 31.631677865982056
Training Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.03
Validation Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.04
Epoch 2, Time Taken : 32.099266052246094
Total Time: 93.50223469734192


State:
	iteration: 938
	epoch: 2
	epoch_length: 469
	max_epochs: 2
	output: 0.007209857925772667
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

## Event-based profiling using `BasicTimeProfiler`

If we want more information without writing custom code for everything via `Timer()`, we can use [`BasicTimeProfiler()`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.time_profilers.BasicTimeProfiler.html#basictimeprofiler) to get the time taken by data processing, data loading and all pre-defined events.

In [54]:
# Attach basic profiler
basic_profiler = BasicTimeProfiler()
basic_profiler.attach(trainer)

trainer.run(train_loader, max_epochs=2)

Training Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.03
Validation Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.04
Epoch 1, Time Taken : 31.97310996055603
Training Results - Epoch[2] Avg accuracy: 1.00 Avg loss: 0.01
Validation Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.03
Epoch 2, Time Taken : 32.139504194259644
Total Time: 95.06597590446472


State:
	iteration: 938
	epoch: 2
	epoch_length: 469
	max_epochs: 2
	output: 0.11615534871816635
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

We can then obtain the results dictionary via [`get_results()`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.time_profilers.BasicTimeProfiler.html#ignite.contrib.handlers.time_profilers.BasicTimeProfiler.get_results) and pass it to [`print_results()`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.time_profilers.BasicTimeProfiler.html#ignite.contrib.handlers.time_profilers.BasicTimeProfiler.print_results) to get a nicely formatted output which contains total, minimum, maximum, mean and the standard deviation of the time taken.

In [55]:
results = basic_profiler.get_results()
basic_profiler.print_results(results)


 ----------------------------------------------------
| Time profiling stats (in seconds):                 |
 ----------------------------------------------------
total  |  min/index  |  max/index  |  mean  |  std

Processing function:
47.88717 | 0.04191/468 | 0.05614/469 | 0.05105 | 0.00086

Dataflow:
16.09671 | 0.01373/467 | 0.03128/694 | 0.01716 | 0.00170

Event handlers:
30.96817

- Events.STARTED: ['Timer.reset']
0.00001

- Events.EPOCH_STARTED: []
0.00001 | 0.00000/1 | 0.00000/0 | 0.00000 | 0.00000

- Events.ITERATION_STARTED: []
0.00228 | 0.00000/234 | 0.00002/597 | 0.00000 | 0.00000

- Events.ITERATION_COMPLETED: []
0.00383 | 0.00000/652 | 0.00003/265 | 0.00000 | 0.00000

- Events.EPOCH_COMPLETED: ['log_training_results', 'log_validation_results', 'Timer.step', 'log_epoch_time']
30.95267 | 14.91956/1 | 16.03311/0 | 15.47634 | 0.78739

- Events.COMPLETED: ['Timer.pause', 'log_total_time']
0.00004



"\n ----------------------------------------------------\n| Time profiling stats (in seconds):                 |\n ----------------------------------------------------\ntotal  |  min/index  |  max/index  |  mean  |  std\n\nProcessing function:\n47.88717 | 0.04191/468 | 0.05614/469 | 0.05105 | 0.00086\n\nDataflow:\n16.09671 | 0.01373/467 | 0.03128/694 | 0.01716 | 0.00170\n\nEvent handlers:\n30.96817\n\n- Events.STARTED: ['Timer.reset']\n0.00001\n\n- Events.EPOCH_STARTED: []\n0.00001 | 0.00000/1 | 0.00000/0 | 0.00000 | 0.00000\n\n- Events.ITERATION_STARTED: []\n0.00228 | 0.00000/234 | 0.00002/597 | 0.00000 | 0.00000\n\n- Events.ITERATION_COMPLETED: []\n0.00383 | 0.00000/652 | 0.00003/265 | 0.00000 | 0.00000\n\n- Events.EPOCH_COMPLETED: ['log_training_results', 'log_validation_results', 'Timer.step', 'log_epoch_time']\n30.95267 | 14.91956/1 | 16.03311/0 | 15.47634 | 0.78739\n\n- Events.COMPLETED: ['Timer.pause', 'log_total_time']\n0.00004\n"

Although this approach does not get the time taken by an individual handler rather the sum of the time taken by all handlers corresponding to a pre-defined event.

## Handler-based profiling using `HandlersTimeProfiler`

We can overcome the above problem by using [`HandlersTimeProfiler`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.time_profilers.HandlersTimeProfiler.html#handlerstimeprofiler) which gives us only the necessary information. We can also calculate the time taken by handlers attached to [`Custom Events`](https://pytorch.org/ignite/concepts.html#custom-events), which was not previously possible, via this.

In [56]:
# Attach handlers profiler
handlers_profiler = HandlersTimeProfiler()
handlers_profiler.attach(trainer)

In [57]:
trainer.run(train_loader, max_epochs=2)

Training Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.03
Validation Results - Epoch[1] Avg accuracy: 0.99 Avg loss: 0.05
Epoch 1, Time Taken : 32.59071087837219
Training Results - Epoch[2] Avg accuracy: 1.00 Avg loss: 0.01
Validation Results - Epoch[2] Avg accuracy: 0.99 Avg loss: 0.03
Epoch 2, Time Taken : 32.15104365348816
Total Time: 95.28053855895996


State:
	iteration: 938
	epoch: 2
	epoch_length: 469
	max_epochs: 2
	output: 0.00042030992335639894
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

We can print the results of the profiler in the same way as above. The output shows total, average and other details of execution time for each handler attached. It also shows the data processing and data loading times.

In [58]:
results = handlers_profiler.get_results()
handlers_profiler.print_results(results)


---------------------------------------  -------------------  --------------  --------------  --------------  --------------  --------------  
Handler                                  Event Name                 Total(s)      Min(s)/IDX      Max(s)/IDX         Mean(s)          Std(s)  
---------------------------------------  -------------------  --------------  --------------  --------------  --------------  --------------  
log_training_results                     EPOCH_COMPLETED            26.26519      12.53628/1      13.72892/0         13.1326         0.84332  
log_validation_results                   EPOCH_COMPLETED             4.27147       2.00636/1       2.26511/0         2.13574         0.18296  
log_epoch_time                           EPOCH_COMPLETED               7e-05         3e-05/1         4e-05/0           3e-05             0.0  
BasicTimeProfiler._as_first_started      STARTED                     0.00071       0.00071/0       0.00071/0         0.00071            None 

The profiling results obtained by `basic_profiler` and `handler_profiler` can be exported to a CSV file by using the `write_results()` method.

In [59]:
basic_profiler.write_results("./basic_profile.csv")
handlers_profiler.write_results("./handlers_profile.csv")

If we inspect the CSV file of `basic_profiler` we can see the depth of information stored for every iteration.

In [60]:
basic_profile = pd.read_csv("./basic_profile.csv")
basic_profile.head()

,epoch,iteration,processing_stats,dataflow_stats,Event_STARTED,Event_COMPLETED,Event_EPOCH_STARTED,Event_EPOCH_COMPLETED,Event_ITERATION_STARTED,Event_ITERATION_COMPLETED,Event_GET_BATCH_STARTED,Event_GET_BATCH_COMPLETED
0,1.0,1.0,0.056409,0.017553,0.000021,0.000088,0.000004,15.994114,0.000004,0.000010,0.000006,0.000012
1,1.0,2.0,0.056446,0.021092,0.000021,0.000088,0.000004,15.994114,0.000004,0.000009,0.000006,0.000012
2,1.0,3.0,0.049232,0.017341,0.000021,0.000088,0.000004,15.994114,0.000004,0.000008,0.000005,0.000011
3,1.0,4.0,0.051980,0.016200,0.000021,0.000088,0.000004,15.994114,0.000004,0.000008,0.000006,0.000012
4,1.0,5.0,0.050854,0.019154,0.000021,0.000088,0.000004,15.994114,0.000004,0.000008,0.000006,0.000016


The `handlers_profile` CSV stores the details for whenever a handler was evoked which corresponds to the number of rows. 

In [61]:
handlers_profile = pd.read_csv("./handlers_profile.csv")
handlers_profile.head()

,#,processing_stats,dataflow_stats,log_training_results (EPOCH_COMPLETED),log_validation_results (EPOCH_COMPLETED),log_epoch_time (EPOCH_COMPLETED),BasicTimeProfiler._as_first_started (STARTED),log_total_time (COMPLETED)
0,1.0,0.056478,0.030867,13.728917,2.265111,0.000035,0.000714,0.000029
1,2.0,0.056523,0.017528,12.536277,2.006360,0.000031,0.000000,0.000000
2,3.0,0.049284,0.021065,0.000000,0.000000,0.000000,0.000000,0.000000
3,4.0,0.052043,0.017314,0.000000,0.000000,0.000000,0.000000,0.000000
4,5.0,0.050914,0.016173,0.000000,0.000000,0.000000,0.000000,0.000000
